# Machine Leaning Exercise 4: Multi-class Classification and Neural Networks

From Week 5 of Coursera course, Machine Learning by Andrew Ng: https://www.coursera.org/learn/machine-learning/. The topic is back-propagation.

Eric Nam, https://github.com/eric-nam, 2020

## Read in the dataset from a MATLAB file
The dataset contains 5,000 hand-written images in rows of a matrix (X) and labels in a vector (y). Each row is an unrolled 20 by 20 image. This data set is the same one from the previous Exercise 3 of Week 4.

In [ ]:
import MAT
fpath_mat = "../week4/ex3data1.mat"
file = MAT.matopen(fpath_mat)
X = MAT.read(file, "X")
y = vec(MAT.read(file, "y"))
MAT.close(file)

## Plot 100 random examples
The code is copied from the previous exercise.

In [ ]:
using Plots
using StatsBase
n = 10
rows = sample(1:size(X)[1], n * n, replace=false)  # Sample 100 randomly
px = isqrt(size(X)[2])
# Combine into one big image
image = zeros(px * n, px * n)
for i in 0:n-1
    for j in 0:n-1
        image[i * px + 1: (i + 1) * px, j * px + 1: (j + 1) * px] = X[rows[i * n + j + 1], :]
    end
end
heatmap(image[end:-1:1, :], c=cgrad(:grays))   # Needs to be flipped upside down

## Read the weights of neurons (neural network parameters) from the file
The weights are the same from the previous exercise.

In [ ]:
fpath_mat = "../week4/ex3weights.mat"
file = MAT.matopen(fpath_mat)
println(MAT.names(file))
theta1 = MAT.read(file, "Theta1")
theta2 = MAT.read(file, "Theta2")
MAT.close(file)

## Unroll the neural network parameters

In [ ]:
theta = vcat(theta1[:], theta2[:]);

## Define a cost function

### Sigmoid function

In [ ]:
"""
    sigmoid(z)

Calculate the sigmoid function, ``g(z) = \\frac{1}{1 + e^{-z}}``

# Argument
- `z::Number`: input variable

# Return
`Number`
"""
function sigmoid(z)
    1.0 / (1.0 + exp(-z))
end

### Cost function
Unlike the original exercise, this function does not return the gradient.

In [ ]:
function cost_function(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda)
    # Recover neural network paramters
    n_theta1 = hidden_layer_size * (input_layer_size + 1)
    theta1 = reshape(nn_params[1:n_theta1], hidden_layer_size, input_layer_size + 1)
    theta2 = reshape(nn_params[n_theta1+1:end], num_labels, (hidden_layer_size + 1))
    m = size(X)[1]
    
    a2 = sigmoid.([ones(m, 1) X] * theta1')
    a3 = sigmoid.([ones(size(a2)[1], 1) a2] * theta2')
    cost = 0.
    Y = zeros(m, num_labels)
    Y[CartesianIndex.(1:m, convert(Array{Int32}, y))] .= 1.
    cost = (sum((- log.(a3) .* Y .- log.(1.0 .- a3) .* (1.0 .- Y))) +
        lambda * 0.5 * (sum(theta1[:, 2:end] .^ 2) + sum(theta2[:, 2:end] .^ 2))) / m
end

### Define some data dimensions

In [ ]:
input_layer_size  = 400  # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units
num_labels = 10

### Cost example without regularization, $\lambda = 0$
The expected value is 0.287629.

In [ ]:
lambda = 0.
cost_function(theta, input_layer_size, hidden_layer_size, num_labels, X, y, lambda)

### Cost example with regularization, $\lambda = 1$
The expected value is 0.383770.

In [ ]:
lambda = 1.
cost_function(theta, input_layer_size, hidden_layer_size, num_labels, X, y, lambda)

## Define sigmoid gradient
$g'(z) = g(z)(1 - g(z))$

In applications, often $g(z)$ or activation that is calculated already are used.

In [ ]:
function sigmoid_gradient(z)
    sigmoid(z) * (1.0 - sigmoid.(z))
end

### Test the sigmoid graident
The gradient of sigmoid at $z=0$ is 0.25.

In [ ]:
sigmoid_gradient.([-1 -0.5 0 0.5 1])

## Training the neural network
Codes belows tries to optimize the weights of the neural network using the backpropagation.

### Define a function to generate randomly initialized of parameters

In [ ]:
function random_initialize_weights(l_in, l_out)
    epsilon_init = 0.12
    theta = (2. .* rand(l_out, 1 + l_in) .- 1.) .* epsilon_init
end

### Define a gradient function of backpropagation
$\delta^{(i)} = (\Theta^{(i)})^T \delta^{(i+1)} \cdot g'(z^{(i)})$

$\Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)} (a^{(l)})^T$

$\frac{\partial}{\partial \Theta^{(l)}} J(\Theta) = D^{(l)} = \Delta^{(l)} / m$

In [ ]:
function cost_gradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda)
    # Recover neural network paramters
    n_theta1 = hidden_layer_size * (input_layer_size + 1)
    theta1 = reshape(nn_params[1:n_theta1], hidden_layer_size, input_layer_size + 1)
    theta2 = reshape(nn_params[n_theta1+1:end], num_labels, (hidden_layer_size + 1))
    m = size(X)[1]
    
    x_aug = [ones(m, 1) X]
    z2 = x_aug * theta1'
    a2 = sigmoid.(z2)
    a2_aug = [ones(size(a2)[1], 1) a2]
    z3 = a2_aug * theta2'
    a3 = sigmoid.(z3)
    Y = zeros(m, num_labels)
    Y[CartesianIndex.(1:m, convert(Array{Int32}, y))] .= 1.
    delta3 = a3 - Y
    theta2_gradient = mapreduce(i -> delta3[i, :] * a2_aug[i, :]', +, 1:m)
    theta2_gradient[:, 2:end] += lambda .* theta2[:, 2:end]
    theta2_gradient /= m
    delta2 = delta3 * theta2 .* a2_aug .* (1. .- a2_aug)
    theta1_gradient = mapreduce(i -> delta2[i, 2:end] * x_aug[i, :]', +, 1:m)
    theta1_gradient[:, 2:end] += lambda .* theta1[:, 2:end]
    theta1_gradient /= m
    # Unroll thetas
    vcat(theta1_gradient[:], theta2_gradient[:])
end

## Train the model!

In [ ]:
using Optim

In [ ]:
initial_theta1 = random_initialize_weights(input_layer_size, hidden_layer_size)
initial_theta2 = random_initialize_weights(hidden_layer_size, num_labels)
# Unroll parameters
initial_theta = vcat(initial_theta1[:],  initial_theta2[:]);

In [ ]:
# Set the lambda, regularization coefficient
lambda = 1.

In [ ]:
n_iterations = 50
result = optimize(t -> cost_function(t, input_layer_size, hidden_layer_size, num_labels, X, y, lambda),
                  t -> cost_gradient(t, input_layer_size, hidden_layer_size, num_labels, X, y, lambda),
                  initial_theta, Optim.Options(iterations=50), inplace=false)

In [ ]:
theta_opt = Optim.minimizer(result);

## Predict with the result

In [ ]:
function predict(theta, X)
    n_theta1 = hidden_layer_size * (input_layer_size + 1)
    theta1 = reshape(theta[1:n_theta1], hidden_layer_size, input_layer_size + 1)
    theta2 = reshape(theta[n_theta1+1:end], num_labels, (hidden_layer_size + 1))
    h1 = sigmoid.([ones(size(X)[1], 1) X] *  theta1')
    h2 = sigmoid.([ones(size(h1)[1], 1) h1] * theta2')
    map(x -> x[2], argmax(h2, dims=2))
end

In [ ]:
predictions = predict(theta_opt, X);

### Check the accuracy
The accuracy would go over 95% if the model is well optimized.

In [ ]:
accuracy = sum(predictions .== y) / size(X)[1] * 100

## Visualize the hidden layer

In [ ]:
n_theta1 = hidden_layer_size * (input_layer_size + 1)
theta1 = reshape(theta[1:n_theta1], hidden_layer_size, input_layer_size + 1)
# Combine into one big image
n = isqrt(hidden_layer_size)
px = isqrt(input_layer_size)
# Combine into one big image
image = zeros(px * n, px * n)
for i in 0:n-1
    for j in 0:n-1
        image[i * px + 1: (i + 1) * px, j * px + 1: (j + 1) * px] = theta1[i * n + j + 1, 2:end]
    end
end
heatmap(image[end:-1:1, :], c=cgrad(:grays))   # Needs to be flipped upside down